<h3>Дисклеймер</h3>

Всё что написано в этом документе является лишь упрощённым объяснением синтаксических контрукций языка и написано таким образом, чтобы помочь вам выполнить домашнее задание. Точные и выверенные формулировки для ответов на зачёте\экзамене рекомендуется брать из материалов лекций и официальной документации языка программирования.

<h1>Декораторы</h1>

Декоратор - паттерн проектирования, который позволяет добавлять некоторую функциональность к различным объектам без изменения самих этих объектов, применения наследования и т.д.
В Питоне декоратор чаще всего является функцией, которая принимает в параметрах функцию (а иногда классом), создаёт внутри себя новую функцию и возвращает её.

Помните, что функции в Питоне - это тоже объекты, как и другие объекты, их можно передать в другие функции, создавать внутри функций и возвращать из функций. По сути, декоратор, реализует некоторую функциональность, которую мы хотим добавить сразу к нескольким объектам, но чтобы не писать этот код несколько раз, мы пишем его единожды (в декораторе), а потом добавляем созданный декоратор в качестве "обёртки" над этими объектами (чаще всего функциями).

Классический декоратор это некая функция X, которая должна принять на вход изначальную функцию A. Создать внутри функцию B, которая будет реализовывать необходимую функциональность и вызывать в процессе своей работы функцию A. Ну а в конце работы X вернёт B.
Посмотрим как это работает, создадим две функции: tsum и tmul - одна складывает два числа, другая, соответственно, умножает.

In [1]:
def tsum(a, b):
    return a + b


def tmul(a, b):
    return a * b


print(tsum(3, 2))
print(tmul(3, 2))

5
6


А теперь представим, у нас в проекте очень много фукнций, они все разные и поступило требование к изменению работы программы "Нужно добавить логирование вызовов всех функций". Казалось бы, нам нужно пойти в каждую функцию и добавить туда код для логирования, но на самом деле нет, мы можем сделать простой логирующий декоратор:

In [2]:
def logger(fn):
    def wrapper(*args, **kwargs):
        print('Func: ' + str(fn))
        print('Args: ' + str(args))
        print('Fwargs: ' + str(kwargs))
        return fn(*args, **kwargs)

    return wrapper

А теперь "навесим" данный декоратор на наши функции (я перепишу их определение ниже, т.к. это удобно с точки зрения чтения iPython Notebook, но вообще в коде достаточно к уже имеющимся определениям функции добавить декоратор).

In [3]:
@logger
def tsum(a, b):
    return a + b


@logger
def tmul(a, b):
    return a * b


print(tsum(3, 2))
print()
print(tmul(3, 2))

Func: <function tsum at 0x103a78220>
Args: (3, 2)
Fwargs: {}
5

Func: <function tmul at 0x103a78040>
Args: (3, 2)
Fwargs: {}
6


А теперь представьте, что поступило изменение к техническому заданию программы и все ответы функций нужно выводить в виде строк, начинающихся с "result is:".

Напишите такой декоратор сами <b>(2 балла)</b>

In [4]:
def result_modifyer(fn):
    def wrapper(*args, **kwargs):
        res = fn(*args, **kwargs)
        return "Result is: " + str(res)

    return wrapper

Теперь проверим, работает ли наше решение <b>(здесь и далее код тестов менять запрещено)</b>:

In [5]:
@result_modifyer
def tsum(a, b):
    return a + b


@result_modifyer
def tmul(a, b):
    return a * b


assert (tsum(2, 3) == "Result is: 5")
assert (tmul(2, 3) == "Result is: 6")
print('Отлично, это работает! +2 балла тебе!')

Отлично, это работает! +2 балла тебе!


Проведём ещё один эксперимент, целью которого является узнать, в каком порядке вызовутся декораторы.

In [6]:
def A(fn):
    def wrapper(*args, **kwargs):
        print('A_1')
        result = fn(*args, **kwargs)
        print('A_2')
        return result

    return wrapper


def B(fn):
    def wrapper(*args, **kwargs):
        print('B_1')
        result = fn(*args, **kwargs)
        print('B_2')
        return result

    return wrapper


@A
@B
def tsum(a, b):
    return a + b


@B
@A
def tmul(a, b):
    return a * b


print(tsum(3, 2))
print()
print(tmul(3, 2))

A_1
B_1
B_2
A_2
5

B_1
A_1
A_2
B_2
6


Как видим, что сначала вызывается "внешний" декоратор (более верхний в коде).

Т.е. сначала вызовется A, из него вызовется B, а уже из него изначальная функция. "Разворачивание" результата пойдёт в обратном порядке.

А теперь представим, что у нас в функции производятся достаточно сложные вычисления и данные функции часто вызываются с одинаковыми параметрами. Что же делать и как оптимизировать?<br>
Написать кеширующий декоратор! Который будет запоминать предыдущие результаты функций и возвращать их из кеша, если функция вновь вызвана с теми же параметрами.

In [ ]:
def cacher(fn):
    cache = {}

    def wrapper(*args, **kwargs):
        str_params = str(args) + str(kwargs)

        if str_params not in cache:
            print("New calc!")
            cache[str_params] = fn(*args, **kwargs)

        return cache[str_params]

    return wrapper

Как мы видим наш декоратор проверяет, есть ли значение в кеше, если нет -> требуется вызвать функцию и дополнить кеш.

Дальше мы просто возвращаем значение из кеша.
Применён небольшой трюк с переводом параметров в строку, связано это с тем, что параметры, с которыми вызывается функция, содержатся в изменяемых объектах. Поэтому они не могут являться ключами в словаре (cache), конвертируем их в строки, строки неизменяемые и всё становится хорошо (почти всегда, но есть исключения ;) ).

Посмотрим как это работает:

In [8]:
@cacher
def tsum(a, b):
    return a + b


print(tsum(3, 2))
print(tsum(3, 2))
print(tsum(3, 3))

New calc!
5
5
New calc!
6


Как вы уже поняли, логирующие декораторы незаменимы, когда требуется на время отладки вывести параметры функции. Вы просто добавляете строчку перед функцией и получаете все параметры, с которыми эту функцию вызывают во время работы программы.

Также декораторы бывают неожиданно полезны, когда вы пишите, например, веб-сервера. Представьте, у вас есть набор веб-страниц, которые доступны только авторизованным пользователям. Можно в начале каждой функции формирования веб-страницы добавить код, проверяющий сессию пользователя. Но зачем, если это можно вынести в декоратор?

Далее рассмотрим класс-декоратор (не путать с декоратором на класс). Вам могло показаться, что в качестве декоратора можно использовать только функцию. Это не так. В качестве декоратора может выступать любой объект, который можно «вызвать». Например, в качестве декоратора может выступать класс. Вот пример, показывающий, как можно конструировать потоки (threads) при помощи декораторов:
(здесь и далее частично использованы примеры: https://habrahabr.ru/post/46306/)

Как видим, значение действительно кешируется. Ура!

Но есть проблема, такая проблема всегда есть с кешами. Размер нашего словаря будет расти пока не кончится память, это плохо, в таких случаях размер кеша как-либо ограничивают.

Сделаем простой вариант, будем кешировать первые size запросов, а размер кеша size будем передавать как параметр декоратора. Да, так можно, декоратор же тоже функция. Почему бы не сделать декоратор с параметром? <b>(4 балла)</b>

Чтобы понять, как должна работать функция в тех случах, когда произошёл "промах" мимо кеша - посмотрите тесты ;)

Подсказка 1: в Питоне можно объявлять функцию внутри функции, намёк вам оставлен в имени возвращаемого значения.

Подсказка 2: в этот раз мы ожидаем, что исходная функция начнёт возвращать строки, так просто было проще писать тесты.

In [19]:
def sized_cacher(size):
    def decor(fn):
        cashe = {}

        def cacher(*args, **kwargs):
            str_args = str(args) + str(kwargs)
            if str_args not in cashe and (len(cashe) < size):
                cashe[str_args] = str(fn(*args, **kwargs))
                return cashe[str_args]
            elif str_args in cashe:
                return "cached: " + str(cashe[str_args])
            else:
                return str(fn(*args, **kwargs))

        return cacher

    return decor


@sized_cacher(2)
def tsum(a, b):
    return a + b


assert (tsum(3, 2) == '5')
assert (tsum(3, 2) == 'cached: 5')
assert (tsum(2, 2) == '4')
assert (tsum(2, 2) == 'cached: 4')
assert (tsum(4, 4) == '8')
assert (tsum(4, 4) == '8')
print('Сразу +4 балла! Not bad!')

Сразу +4 балла! Not bad!


Как мы видим, мы задали размер кеша равный 2, действительно, первые два результата закешировались и больше не пересчитывается, а остальные будут считаться каждый раз.

Но наш кеш довольно странный, он хранит результаты первых size вызовов. На самом деле есть разные стратегии к построению кеша, например, можно кешировать результаты для самых часто вызываемых аргументов. Или можно кешировать некоторое количество последних вызовов. Какую именно стратегию выбвать для построения кеша необходимо решить глядя на задачу, которую вы решаете, например, если велика вероятность, что вызовы с одинаковыми параметрами будут производиться "друг за другом" будет хорошо кешировать сколько-нибудь последних вызовов, такая стратегий называется least recently used или LRU.

В стандартной поставке Python уже есть реализация такого кеширующего декоратора:

In [20]:
from functools import lru_cache


@lru_cache(maxsize=1)
def tsum(a, b):
    print("func call! Not cache!")
    return a + b


print(tsum(3, 2))
print(tsum(3, 2))
print(tsum(2, 2))
print(tsum(2, 2))
print(tsum(3, 2))

func call! Not cache!
5
5
func call! Not cache!
4
4
func call! Not cache!
5


Видно, что если мы задали размер кеша 1, сначала у нас закешировался первый вызов, потом их вытеснили из кеша другие аргументы и т.д. Для ещё одного примера применения декораторов представим, что мы написали программу, но она работает долго. Мы хотим узнать, какая функция работает дольше всех и где именно у нас просадки по производительности. Нет ничего проще! Будем считать время выполнения функций: <b>(2 балла)</b>

Чтобы понять в каком формате вывод функции от нас ждут можно снова посмотреть на тесты и в docstrings декоратора.

In [21]:
from time import time, sleep


def timer(fn):
    '''Возвращает tuple, содержащий время выполнения функции и результат'''

    def wrapper(*args, **kwargs):
        start = time()
        result = fn(*args, **kwargs)
        end = time()
        end_time = end - start
        return (end_time, result)

    return wrapper


@timer
def tsum(a, b):
    sleep(1)
    return a + b


@timer
def tmul(a, b):
    sleep(2)
    return a * b


result_1 = tsum(2, 3)
result_2 = tmul(2, 3)

assert (result_1[0] > 1)
assert (result_1[1] == 5)
assert (result_2[0] > 2)
assert (result_2[1] == 6)
print('Успешно задекорировано! Ещё +2 балла!')

Успешно задекорировано! Ещё +2 балла!


Как вы уже поняли, логирующие декораторы незаменимы, когда требуется на время отладки вывести параметры функции. Вы просто добавляете строчку перед функцией и получаете все параметры, с которыми эту функцию вызывают во время работы программы.

Также декораторы бывают неожиданно полезны, когда вы пишите, например, веб-сервера. Представьте, у вас есть набор веб-страниц, которые доступны только авторизованным пользователям. Можно в начале каждой функции формирования веб-страницы добавить код, проверяющий сессию пользователя. Но зачем, если это можно вынести в декоратор?

Далее рассмотрим класс-декоратор (не путать с декоратором на класс). Вам могло показаться, что в качестве декоратора можно использовать только функцию. Это не так. В качестве декоратора может выступать любой объект, который можно «вызвать». Например, в качестве декоратора может выступать класс. Вот пример, показывающий, как можно конструировать потоки (threads) при помощи декораторов:
(здесь и далее частично использованы примеры: https://habrahabr.ru/post/46306/)

In [22]:
import threading


class Thread(threading.Thread):
    def __init__(self, f):
        threading.Thread.__init__(self)
        self.run = f
        self.start()


@Thread
def ttt():
    print(threading.current_thread().ident)


print(threading.current_thread().ident)

63052472327946080256



Если выше вы видите одно непонятное длинное число - значит два потока "подрались" на вывод на экран и их id "смешались", если позапускать ячейку несколько раз, этот race condition (если не знаешь что это такое срочно гугли!) должен уйти и хотя бы раз ты увидишь два разных id, что позволит нам удостовериться, что это разные потоки.

Использование декораторов на методах классов ничем не отличается от использования декораторов на обычных функциях.

Однако для классов есть предопределённые декораторы с именами staticmethod и classmethod. Они предназначены для задания статических методов и методов класса соответственно. Вот пример их использования:

In [23]:
class TestClass(object):
    @classmethod
    def f1(cls):
        print('Это метод, который вместо экземпляра объекта (self), получает экземпляр класса (cls)')
        print(cls.__name__)

    @staticmethod
    def f2():
        print('Это f2. Здесь у нас нет доступа к полям и методам объекта, self не передаётся')
        print('Это более безопасно, чем обычный метод')


class TestClass2(TestClass):
    pass


TestClass.f1()  # печатает TestClass
TestClass2.f1()  # печатает TestClass2

print()
TestClass2.f2()

Это метод, который вместо экземпляра объекта (self), получает экземпляр класса (cls)
TestClass
Это метод, который вместо экземпляра объекта (self), получает экземпляр класса (cls)
TestClass2

Это f2. Здесь у нас нет доступа к полям и методам объекта, self не передаётся
Это более безопасно, чем обычный метод


А теперь посмотрим как выглядит декоратор на класс, не путать с классом-декоратором (он декорирует класс, а класс-декоратор может декорировать функцию, см. пример с потоками выше):

In [24]:
def id_decorator(cls):
    old_init = cls.__init__

    def new_init(self, id, *args, **kwargs):
        # можем навесить на декорируемый класс свои поля и методы, сделать универсальный интерфейс        
        def get_id():
            return self._id

        self._id = id
        self.get_id = get_id
        old_init(self, *args, **kwargs)

    # наш декоратор на класс умеет подменять оригинальную функцию init у класса, который он декорирует :)
    cls.__init__ = new_init
    return cls


@id_decorator
class TestClass:
    def __init__(self, value):
        self.value = value

    def get_value(self):
        return self.value


object1 = TestClass(1, "First")
object2 = TestClass(2, "Second")
print(object1.get_value(), object1.get_id())
print(object2.get_value(), object2.get_id())

First 1
Second 2


Ну а теперь научимся писать синглтоны на Python проще, чем мы это делали раньше.

Каждый специалист по компьютерным наукам должен быть знаком с этим паттерном: <a href="https://tinyurl.com/uwyz4aww">ПРОЧТИ МЕНЯ!!!</a>

In [25]:
def singleton(cls):
    instances = {}

    def getinstance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]

    return getinstance


@singleton
class Foo(object):
    def bar(self):
        pass


print(id(Foo()))
print(id(Foo()))

4356306608
4356306608


Как видим, Id объектов совпадают, значит, созданные объекты совпадают (являются одним и тем же объектом).

Что делать если у нашей функции есть различные метаданные (например, docstrings) и мы хотим их сохранить, пропустив функцию через декоратор? Это возможно:

In [26]:
from functools import wraps


def bad_decorator(fn):
    def wrapper(*args, **kwargs):
        return fn(*args, **kwargs)

    return wrapper


def good_decorator(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        return fn(*args, **kwargs)

    return wrapper


@bad_decorator
def test_func(a, b):
    """function(a, b) -> list"""
    return [a, b]


# также можно использовать help(test_func) для просмотра docstrings ;)
print(test_func.__doc__)


@good_decorator
def test_func(a, b):
    """function(a, b) -> list"""
    return [a, b]


print(test_func.__doc__)

None
function(a, b) -> list


Как видим, functools.wraps помог решить проблему потери метаданных функции, в частности, потерю docstrings.

Теперь рассмотрим ещё один встроенный декоратор @property:

In [17]:
class Test:
    @property
    def x(self):
        """I'm the property."""
        return 1 * 3


test = Test()
print(test.x)

3


Мы привыкли, что если у нас класс генерирует какое-то значение - нужен метод, ведь нужен код, который может генерировать это значение.

Оказывается, можно представлять это значение (генерируемое динамически) для внешнего мира как поле объекта при помощи декоратора @property.

Ух ты, ты правда всё это прочитал? Тогда доделай немного код ниже и получи за это ещё +2 балла :)

In [27]:
def get_gift(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        print('Я молодец! Я заслужил +2 балла!')
        return fn(*args, **kwargs)

    return wrapper


@get_gift
def last_func():
    print('Ура, на сегодня всё! Спасибо за внимание!')


last_func()

Я молодец! Я заслужил +2 балла!
Ура, на сегодня всё! Спасибо за внимание!


Версия документа 1.3

© Вячеслав Копейцев, 2024

P.S. Автор не претендует на некую "экспертизу" в языке Python, код и текст могут содержать ошибки.